In [2]:
from molytica_m.data_tools import PPI_interactome_tools
from molytica_m.data_tools import alpha_fold_tools

2023-12-11 20:49:51.488987: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-11 20:49:51.521429: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-11 20:49:51.521462: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-11 20:49:51.521496: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-11 20:49:51.527832: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-11 20:49:51.528195: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

cuda


In [16]:
af_uniprots = alpha_fold_tools.get_alphafold_uniprot_ids()
PPI_interactome_tools.get_ppi_prob(af_uniprots[-1], af_uniprots[-2])

0.021102964412420988

In [15]:
PPI_interactome_tools.get_ppi_prob("Q9H9S0", "Q01860")

In [5]:
210/0.4*26.5/1000

13.9125

In [14]:
x1 = 6165000
x3 = 210217000 - 2151000
x2 = (x3 - x1) * 1/3 + x1
x2

73465333.33333333

In [6]:
from molytica_m.data_tools import create_PPI_dataset
from molytica_m.data_tools import alpha_fold_tools
from tqdm import tqdm
import numpy as np
import torch
import h5py
import os
import sqlite3

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("cuda" if torch.cuda.is_available() else "cpu")
model = torch.load("molytica_m/ml/PPI_S_model.pth", map_location=torch.device(device)).to(device)
model.eval()

cuda


ProteinInteractionPredictor(
  (fc1): Linear(in_features=18640, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=128, bias=True)
  (gcn1): GCNConv(9, 128)
  (gcn2): GCNConv(128, 128)
  (fc_combined): Linear(in_features=512, out_features=128, bias=True)
  (output): Linear(in_features=128, out_features=1, bias=True)
)

In [7]:
import random
uniprot_pairs = [[random.choice(af_uniprots), random.choice(af_uniprots)], [random.choice(af_uniprots), random.choice(af_uniprots)]]

In [1]:
metadata_as = []
metadata_bs = []

x_as, edge_index_as = [], []
x_bs, edge_index_bs = [], []


for uniprot_pair in uniprot_pairs:
    uniprot_A = uniprot_pair[0]
    uniprot_B = uniprot_pair[1]

    metadata_a = create_PPI_dataset.get_metadata(uniprot_A)
    metadata_b = create_PPI_dataset.get_metadata(uniprot_B)
    
    x_a, edge_index_a = create_PPI_dataset.get_graph_raw(uniprot_A)
    x_b, edge_index_b = create_PPI_dataset.get_graph_raw(uniprot_B)

    metadata_as.append(metadata_a)
    metadata_bs.append(metadata_b)
    
    x_as.append(x_a)
    edge_index_as.append(edge_index_a)
    x_bs.append(x_b)
    edge_index_bs.append(edge_index_b)


def pad_arrays(arrays, pad_value=0):
    max_shape = max((a.shape for a in arrays), key=lambda x: (x[0], x[1]))
    padded_arrays = [np.pad(a, ((0, max_shape[0] - a.shape[0]), (0, max_shape[1] - a.shape[1])), 
                            mode='constant', constant_values=pad_value) for a in arrays]
    return padded_arrays
    
padded_x_as = np.array(pad_arrays(x_as))
padded_x_bs = np.array(pad_arrays(x_bs))
padded_edge_index_as = np.array(pad_arrays(edge_index_as)) 
padded_edge_index_bs = np.array(pad_arrays(edge_index_bs))

# Print the full array
print(padded_edge_index_as[1][1])


model_input = []

(
torch.tensor(metadata_as, dtype=torch.float).to(device),
torch.tensor(metadata_bs, dtype=torch.float).to(device),
torch.tensor(padded_x_as, dtype=torch.float).to(device),
torch.tensor(padded_edge_index_as, dtype=torch.long).to(device),
torch.tensor(padded_x_bs, dtype=torch.float).to(device),
torch.tensor(padded_edge_index_bs, dtype=torch.long).to(device),
)

model(*model_input)

NameError: name 'uniprot_pairs' is not defined

In [17]:
import torch_geometric